In [17]:
import pandas as pd
import geopandas as gpd
import skmob
from skmob.tessellation import tilers
from shapely import wkt, MultiPolygon

In [18]:
column_names = ['userid', 'timecome','date','lat','long','count','timeleave','duration']
data = pd.read_csv('../../data/raw/stay_points_07/2017-07-03.txt', names=column_names)

In [19]:
to_tdf = data[['userid', 'lat', 'long', 'timecome']]
tdf = skmob.TrajDataFrame(to_tdf, latitude='lat', longitude='long', datetime='timecome', user_id='userid')
tdf.crs

{'init': 'epsg:4326'}

In [20]:
tdf_test = tdf[0:10000]

In [21]:
bg_shapes = pd.read_csv('../../data/processed/bg_shapefile.csv')
bg_shapes = gpd.GeoDataFrame(bg_shapes, geometry=bg_shapes['geometry'].apply(wkt.loads))

print(bg_shapes.shape)
bg_shapes = bg_shapes.drop(bg_shapes[bg_shapes['geometry'].apply(lambda geom: isinstance(geom, MultiPolygon))].index)
print(bg_shapes.shape)

bg_shapes.reset_index(inplace=True, drop=True)

tessellation_columns = ['GEOID', 'geometry']
bg_tessellation = bg_shapes[tessellation_columns]
bg_tessellation.rename(columns={'GEOID':'tile_ID'}, inplace=True)


bg_tessellation = bg_tessellation.set_crs('epsg:4326')
bg_tessellation.head()

(4152, 12)
(4143, 12)


/tmp/ipykernel_9897/3141784216.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bg_tessellation.rename(columns={'GEOID':'tile_ID'}, inplace=True)


,tile_ID,geometry
0,483396906052,"POLYGON ((-95.56428 30.18810, -95.56368 30.192..."
1,483396933031,"POLYGON ((-95.48293 30.28567, -95.48143 30.288..."
2,483396911002,"POLYGON ((-95.51639 30.17621, -95.51416 30.178..."
3,483396931022,"POLYGON ((-95.45590 30.29504, -95.45300 30.295..."
4,483396920092,"POLYGON ((-95.39666 30.10288, -95.39003 30.108..."


In [22]:
flow, tessellation = tdf.to_flowdataframe(tessellation=bg_tessellation, self_loops=True)
flow = pd.DataFrame(flow)
flow.origin = flow.origin.astype('int64')
flow.destination = flow.destination.astype('int64')
flow.head()

/home/caden/anaconda3/envs/ai/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/caden/Programming/scikit-mobility/skmob/core/trajectorydataframe.py:274: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  flow = self.mapping(tessellation, remove_na=False)
/home/caden/Programming/scikit-mobility/skmob/core/trajectorydataframe.py:375: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  tile_i

,origin,destination,flow
0,480157601001,480157601001,199
1,480157601001,480157601002,9
2,480157601001,480157602011,1
3,480157601001,480157602021,3
4,480157601001,480157602022,3


In [23]:
fdf = skmob.FlowDataFrame(data=flow, tessellation=tessellation)

In [24]:
fdf.shape

(326644, 3)

In [25]:
fdf1.shape

(256119, 3)

In [26]:
combined_df = pd.concat([fdf, fdf1], ignore_index=True)


In [27]:
combined_df = combined_df.groupby(['origin', 'destination']).sum().reset_index()

In [29]:
combined_df.origin.nunique()

4142

In [32]:
combined_df

,origin,destination,flow
0,480157601001,480157601001,383
1,480157601001,480157601002,18
2,480157601001,480157602011,1
3,480157601001,480157602021,5
4,480157601001,480157602022,5
...,...,...,...
482699,484739800001,484736803032,2
482700,484739800001,484736805011,1
482701,484739800001,484736805012,2
482702,484739800001,484736805023,1


In [31]:
print(combined_df.shape)
print(fdf.shape)
print(fdf1.shape)

(482704, 3)
(326644, 3)
(256119, 3)
